In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
path = "/content/drive/MyDrive/datas"
os.listdir(path)


['purchase_prices.csv',
 'vendor_invoice.csv',
 'begin_inventory.csv',
 'purchases.csv',
 'end_inventory.csv',
 'sales.csv',
 'logs',
 'inventory (1).db',
 'inventory.db']

In [3]:
log_folder = "/content/drive/MyDrive/datas/logs/"
os.makedirs(log_folder, exist_ok=True)

import logging
# Reset previous logging
for h in logging.root.handlers[:]:
    logging.root.removeHandler(h)

logging.basicConfig(
    filename=log_folder + "ingestion_db.log",
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"
)


In [4]:
db_path = "/content/drive/MyDrive/datas/inventory.db"

# Make sure you don't accidentally overwrite anything
if os.path.exists(db_path) and os.path.getsize(db_path) > 0:
    print("DB already exists and is not empty.")
else:
    print("DB will be created/recreated.")


DB already exists and is not empty.


In [5]:
from sqlalchemy import create_engine

engine = create_engine(f"sqlite:///{db_path}", connect_args={"check_same_thread": False})


In [6]:
import pandas as pd
import time

def ingest_db_chunk(df, table_name, engine, if_exists):
    df.to_sql(table_name, con=engine, if_exists=if_exists, index=False)

def load_raw_data():
    logging.info("CSV Ingestion Started")

    for file in os.listdir(path):
        if not file.endswith(".csv"):
            continue

        csv_path = os.path.join(path, file)
        table_name = file[:-4]
        logging.info(f"Started: {file}")
        print(f"\nIngesting: {file}")

        first_chunk = True
        chunk_index = 0
        start_time = time.time()

        for chunk in pd.read_csv(csv_path, chunksize=250_000, low_memory=False):
            ingest_db_chunk(
                chunk,
                table_name,
                engine,
                if_exists="replace" if first_chunk else "append"
            )
            first_chunk = False
            print(f"  Chunk {chunk_index} done")
            chunk_index += 1

        duration = round((time.time() - start_time)/60, 2)
        logging.info(f"Completed {file} in {duration} minutes")
        print(f"Completed {file} in {duration} minutes")

    logging.info("CSV Ingestion Complete")
    print("\nAll files ingested successfully!")


In [7]:
load_raw_data()



Ingesting: purchase_prices.csv
  Chunk 0 done
Completed purchase_prices.csv in 0.59 minutes

Ingesting: vendor_invoice.csv
  Chunk 0 done
Completed vendor_invoice.csv in 0.01 minutes

Ingesting: begin_inventory.csv
  Chunk 0 done
Completed begin_inventory.csv in 0.14 minutes

Ingesting: purchases.csv
  Chunk 0 done
  Chunk 1 done
  Chunk 2 done
  Chunk 3 done
  Chunk 4 done
  Chunk 5 done
  Chunk 6 done
  Chunk 7 done
  Chunk 8 done
  Chunk 9 done
Completed purchases.csv in 2.46 minutes

Ingesting: end_inventory.csv
  Chunk 0 done
Completed end_inventory.csv in 0.14 minutes

Ingesting: sales.csv
  Chunk 0 done
  Chunk 1 done
  Chunk 2 done
  Chunk 3 done
  Chunk 4 done
  Chunk 5 done
  Chunk 6 done
  Chunk 7 done
  Chunk 8 done
  Chunk 9 done
  Chunk 10 done
  Chunk 11 done
  Chunk 12 done
  Chunk 13 done
  Chunk 14 done
  Chunk 15 done
  Chunk 16 done
  Chunk 17 done
  Chunk 18 done
  Chunk 19 done
  Chunk 20 done
  Chunk 21 done
  Chunk 22 done
  Chunk 23 done
  Chunk 24 done
  Chun

In [8]:
import sqlite3

In [9]:
#creating database connection
conn = sqlite3.connect('/content/drive/MyDrive/datas/inventory.db')

In [10]:
#checking tables present in the database
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)
tables

,name
0,purchase_prices
1,vendor_invoice
2,begin_inventory
3,purchases
4,end_inventory
5,sales


# **Exploratory Data analysis**

In [11]:
#improvised code for Count of records from tables
import sqlite3
import pandas as pd

conn = sqlite3.connect(db_path)

tables = ['begin_inventory', 'end_inventory', 'purchase_prices', 'purchases', 'sales', 'vendor_invoice']

for table in tables:
    query = f"SELECT COUNT(*) AS total FROM {table}"
    count = pd.read_sql_query(query, conn)['total'].values[0]
    print('-'*50, f'{table}', '-'*50)
    print(f"Count of records: {count}\n")



-------------------------------------------------- begin_inventory --------------------------------------------------
Count of records: 206529

-------------------------------------------------- end_inventory --------------------------------------------------
Count of records: 224489

-------------------------------------------------- purchase_prices --------------------------------------------------
Count of records: 12261

-------------------------------------------------- purchases --------------------------------------------------
Count of records: 2372474

-------------------------------------------------- sales --------------------------------------------------
Count of records: 12825363

-------------------------------------------------- vendor_invoice --------------------------------------------------
Count of records: 5543



In [12]:
display(pd.read_sql_query(f"select * from {table} limit 5", conn))

,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,105,ALTAMAR BRANDS LLC,2024-01-04,8124,2023-12-21,2024-02-16,6,214.26,3.47,None
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57,None
2,388,ATLANTIC IMPORTING COMPANY,2024-01-09,8169,2023-12-24,2024-02-16,5,106.60,4.61,None
3,480,BACARDI USA INC,2024-01-12,8106,2023-12-20,2024-02-05,10100,137483.78,2935.20,None
4,516,BANFI PRODUCTS CORP,2024-01-07,8170,2023-12-24,2024-02-12,1935,15527.25,429.20,None


In [13]:
purchase = pd.read_sql_query("select * from purchases where VendorNumber = 388", conn)
purchase

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,76_DONCASTER_2034,76,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8169,2023-12-24,2024-01-02,2024-01-09,2024-02-16,21.32,5,106.60,1
1,42_BLACK HOLLOW_2034,42,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8249,2023-12-30,2024-01-06,2024-01-11,2024-02-18,21.32,30,639.60,1
2,79_BALLYMENA_2034,79,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8249,2023-12-30,2024-01-09,2024-01-11,2024-02-18,21.32,35,746.20,1
3,32_MOUNTMEND_2034,32,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8249,2023-12-30,2024-01-05,2024-01-11,2024-02-18,21.32,6,127.92,1
4,50_MOUNTMEND_2034,50,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8249,2023-12-30,2024-01-06,2024-01-11,2024-02-18,21.32,6,127.92,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,70_KELD_2034,70,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,13624,2024-12-22,2024-12-27,2025-01-06,2025-02-07,21.32,6,127.92,1
262,1_HARDERSFIELD_2034,1,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,13624,2024-12-22,2024-12-31,2025-01-06,2025-02-07,21.32,6,127.92,1
263,32_MOUNTMEND_2034,32,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,13624,2024-12-22,2024-12-27,2025-01-06,2025-02-07,21.32,6,127.92,1
264,59_CLAETHORPES_2034,59,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,13624,2024-12-22,2024-12-28,2025-01-06,2025-02-07,21.32,6,127.92,1


In [14]:
purchase_price = pd.read_sql_query("select * from purchase_prices where VendorNumber = 388", conn)
purchase_price

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,2034,Glendalough Double Barrel,32.99,750mL,750,1,21.32,388,ATLANTIC IMPORTING COMPANY
1,2119,Glendalough 13 Yr Sngle Malt,89.99,750mL,750,1,66.40,388,ATLANTIC IMPORTING COMPANY
2,7241,Glendalough Poitin Shry Cask,34.99,750mL,750,2,20.68,388,ATLANTIC IMPORTING COMPANY


In [15]:
vendor_invoice = pd.read_sql_query("select * from vendor_invoice where VendorNumber = 388", conn)
vendor_invoice

,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,388,ATLANTIC IMPORTING COMPANY,2024-01-09,8169,2023-12-24,2024-02-16,5,106.60,4.61,None
1,388,ATLANTIC IMPORTING COMPANY,2024-01-11,8249,2023-12-30,2024-02-18,95,2025.40,10.73,None
2,388,ATLANTIC IMPORTING COMPANY,2024-01-25,8323,2024-01-04,2024-02-23,64,1364.48,7.10,None
3,388,ATLANTIC IMPORTING COMPANY,2024-01-31,8456,2024-01-13,2024-02-28,18,379.92,1.98,None
4,388,ATLANTIC IMPORTING COMPANY,2024-02-03,8497,2024-01-17,2024-03-08,83,2028.52,9.53,None
5,388,ATLANTIC IMPORTING COMPANY,2024-02-08,8675,2024-01-28,2024-03-13,82,1748.24,9.27,None
6,388,ATLANTIC IMPORTING COMPANY,2024-02-19,8704,2024-01-31,2024-03-20,24,511.68,2.40,None
7,388,ATLANTIC IMPORTING COMPANY,2024-02-25,8893,2024-02-12,2024-03-29,12,255.84,1.18,None
8,388,ATLANTIC IMPORTING COMPANY,2024-02-29,8962,2024-02-17,2024-04-08,29,618.28,3.34,None
9,388,ATLANTIC IMPORTING COMPANY,2024-03-08,9095,2024-02-26,2024-04-20,54,1151.28,6.22,None


In [16]:
sales = pd.read_sql_query("select * from sales where VendorNo = 388", conn)
sales

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_2034,1,2034,Glendalough Double Barrel,750mL,1,28.99,28.99,2024-01-12,750.0,1,0.79,388,ATLANTIC IMPORTING COMPANY
1,1_HARDERSFIELD_2034,1,2034,Glendalough Double Barrel,750mL,1,28.99,28.99,2024-01-13,750.0,1,0.79,388,ATLANTIC IMPORTING COMPANY
2,1_HARDERSFIELD_2034,1,2034,Glendalough Double Barrel,750mL,1,28.99,28.99,2024-01-20,750.0,1,0.79,388,ATLANTIC IMPORTING COMPANY
3,1_HARDERSFIELD_2034,1,2034,Glendalough Double Barrel,750mL,1,28.99,28.99,2024-01-22,750.0,1,0.79,388,ATLANTIC IMPORTING COMPANY
4,10_HORNSEY_2034,10,2034,Glendalough Double Barrel,750mL,1,28.99,28.99,2024-01-20,750.0,1,0.79,388,ATLANTIC IMPORTING COMPANY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,7_STANMORE_2034,7,2034,Glendalough Double Barrel,750mL,1,27.99,27.99,2024-12-21,750.0,1,0.79,388,ATLANTIC IMPORTING COMPANY
1476,81_PEMBROKE_2034,81,2034,Glendalough Double Barrel,750mL,2,55.98,27.99,2024-12-19,750.0,1,1.57,388,ATLANTIC IMPORTING COMPANY
1477,81_PEMBROKE_2034,81,2034,Glendalough Double Barrel,750mL,1,27.99,27.99,2024-12-30,750.0,1,0.79,388,ATLANTIC IMPORTING COMPANY
1478,9_BLACKPOOL_2034,9,2034,Glendalough Double Barrel,750mL,1,27.99,27.99,2024-12-01,750.0,1,0.79,388,ATLANTIC IMPORTING COMPANY


In [17]:
purchase.groupby(['Brand','PurchasePrice'])[['Quantity', 'Dollars']].sum()

,,Quantity,Dollars
Brand,PurchasePrice,,
2034,21.32,1777,37885.64
2119,66.40,39,2589.60
7241,20.68,31,641.08


In [18]:
purchase.columns

Index(['InventoryId', 'Store', 'Brand', 'Description', 'Size', 'VendorNumber',
       'VendorName', 'PONumber', 'PODate', 'ReceivingDate', 'InvoiceDate',
       'PayDate', 'PurchasePrice', 'Quantity', 'Dollars', 'Classification'],
      dtype='object')

In [19]:
sales.columns

Index(['InventoryId', 'Store', 'Brand', 'Description', 'Size', 'SalesQuantity',
       'SalesDollars', 'SalesPrice', 'SalesDate', 'Volume', 'Classification',
       'ExciseTax', 'VendorNo', 'VendorName'],
      dtype='object')

In [20]:
purchase_price.columns

Index(['Brand', 'Description', 'Price', 'Size', 'Volume', 'Classification',
       'PurchasePrice', 'VendorNumber', 'VendorName'],
      dtype='object')

In [21]:
sales.groupby(['Brand'])[['SalesQuantity', 'SalesDollars','SalesPrice']].sum()

,SalesQuantity,SalesDollars,SalesPrice
Brand,,,
2034,1587,45645.13,37387.00
2119,115,9838.85,6924.19
7241,123,3788.77,2989.01




*   purchases table contain actual purchase data including the data of purchase, products(brands) purchased by vendors,the amount paid (in dollars), and the quantity purchased.
*   the purchase price columns is derived from the purchase_price table, which provides products wise actual and purchase price. The combination of vendor and brand is unique in this table.
*   the vendor_invoice table aggregates data from  purchases table, summarizing quantity and dollar amount, along with an additional column for frieght. this A table based on uniqueness based on vendor and PO number.
*   the sales table captures actual sales transection, detailing brand purcahse by vendors, the quantity sold, the selling price, and the revenue earned.




As the data we need to analysis is distributed in different tables, we need to create a summary table containing-


*   purchase transaction made by vendors

*   sales transaction data
*  frieght cost for each vendors


*  actual product price from vendors



In [22]:
freight_summary = pd.read_sql_query("select VendorNumber, sum(Freight) as freight_cost from vendor_invoice group by VendorNumber", conn)
freight_summary

,VendorNumber,freight_cost
0,2,27.08
1,54,0.48
2,60,367.52
3,105,62.39
4,200,6.19
...,...,...
121,98450,856.02
122,99166,130.09
123,172662,178.34
124,173357,202.50


In [23]:
pd.read_sql_query("""Select
                  p.VendorNumber,
                  p.VendorName,
                  p.brand,
                  p.PurchasePrice,
                  pp.Volume,
                  pp.Price as ActualPrice,
                  Sum(p.quantity) as TotalPurchaseQuantity,
                  Sum(p.dollars) as TotalPurchaseDollars
                  From Purchases p
                  Join purchase_prices pp
                  on p.brand = pp.brand
                  where p.PurchasePrice >0
                  Group by p.VendorNumber,
                  p.VendorName,
                  p.brand
                  order by TotalPurchaseDollars Desc""",conn)

,VendorNumber,VendorName,Brand,PurchasePrice,Volume,ActualPrice,TotalPurchaseQuantity,TotalPurchaseDollars
0,1128,BROWN-FORMAN CORP,1233,26.27,1750,36.99,145080,3811251.60
1,4425,MARTIGNETTI COMPANIES,3405,23.19,1750,28.99,164038,3804041.22
2,17035,PERNOD RICARD USA,8068,18.24,1750,24.99,187407,3418303.68
3,3960,DIAGEO NORTH AMERICA INC,4261,16.17,1750,22.99,201682,3261197.94
4,3960,DIAGEO NORTH AMERICA INC,3545,21.89,1750,29.99,138109,3023206.01
...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,1.32,750,4.99,2,2.64
10688,8004,SAZERAC CO INC,5683,0.39,50,0.49,6,2.34
10689,3924,HEAVEN HILL DISTILLERIES,9123,0.74,50,0.99,2,1.48
10690,3960,DIAGEO NORTH AMERICA INC,6127,1.47,200,1.99,1,1.47


In [24]:
pd.read_sql_query("""Select
                  VendorNo,
                  VendorName,
                  Brand,
                  Sum(SalesQuantity) as TotalSalesQuantity,
                  Sum(SalesDollars) as TotalSalesDollars,
                  Sum(ExciseTax) as TotalExciseTax,
                  Sum(SalesPrice) as TotalSalesPrice
                  From sales
                  Group by VendorNo,
                  Brand
                  order by TotalSalesDollars Desc""",conn)

,VendorNo,VendorName,Brand,TotalSalesQuantity,TotalSalesDollars,TotalExciseTax,TotalSalesPrice
0,1128,BROWN-FORMAN CORP,1233,142049,5.101920e+06,260999.20,672819.31
1,4425,MARTIGNETTI COMPANIES,3405,160247,4.819073e+06,294438.66,561512.37
2,17035,PERNOD RICARD USA,8068,187140,4.538121e+06,343854.07,461140.15
3,3960,DIAGEO NORTH AMERICA INC,4261,200412,4.475973e+06,368242.80,420050.01
4,3960,DIAGEO NORTH AMERICA INC,3545,135838,4.223108e+06,249587.83,545778.28
...,...,...,...,...,...,...,...
11267,3252,E & J GALLO WINERY,3933,2,1.980000e+00,0.10,0.99
11268,3924,HEAVEN HILL DISTILLERIES,9123,2,1.980000e+00,0.10,0.99
11269,10050,Russian Standard Vodka,3623,2,1.980000e+00,0.10,1.98
11270,9206,PHILLIPS PRODUCTS CO.,2773,1,9.900000e-01,0.05,0.99


In [25]:
sales.columns

Index(['InventoryId', 'Store', 'Brand', 'Description', 'Size', 'SalesQuantity',
       'SalesDollars', 'SalesPrice', 'SalesDate', 'Volume', 'Classification',
       'ExciseTax', 'VendorNo', 'VendorName'],
      dtype='object')

# NOTE:
# Running this query directly on a very large dataset will be extremely slow
# because it requires heavy aggregation (SUM, GROUP BY) on millions of rows.
# Instead of executing it as-is, we will replace it with a more optimized
# version of the query to improve performance.

```
# This is formatted as code
```import time
start = time.time()
final_table = pd.read_sql_query("""Select
                  pp.VendorNumber,
                  pp.Brand,
                  pp.price as actual price,
                  pp.PurchasePrice,
                  Sum(SalesQuantity) as TotalSalesQuantity,
                  Sum(SalesDollars) as TotalSalesDollars,
                  Sum(SalesPrice) as TotalSalesPrice,
                  Sum(ExciseTax) as TotalExciseTax,
                  sum(vi.quantity) as TotalPurchaseQuantity,
                  sum(vi.dollars) as TotalPurchaseDollars,
                  Sum(vi.Freight) as TotalFreightCost
                  From purchase_prices pp
                  Join sales s
                  on pp.VendorNumber = s.VendorNo
                  and pp.brand = s.Brand
                  Join vendor_invoice vi
                  on pp.VendorNumber = vi.VendorNumber
                  Group by pp.VendorNumber,
                  pp.Brand,
                  pp.price,
                  pp.PurchasePrice,
                  order by TotalSalesDollars Desc""",conn)
end = time.time()



In [26]:
vendor_sales_summary = pd.read_sql("""WITH FreightSummary AS (
    SELECT
        VendorNumber,
        SUM(Freight) AS freight_cost
    FROM vendor_invoice
    GROUP BY VendorNumber
),

PurchaseSummary AS (
    SELECT
        p.VendorNumber,
        p.VendorName,
        p.Brand,
        p.PurchasePrice,
        p.Description,
        pp.Volume,
        pp.Price AS ActualPrice,
        SUM(p.Quantity) AS TotalPurchaseQuantity,
        SUM(p.Dollars) AS TotalPurchaseDollars
    FROM Purchases p
    JOIN purchase_prices pp
        ON p.Brand = pp.Brand
    WHERE p.PurchasePrice > 0
    GROUP BY
        p.VendorNumber,
        p.VendorName,
        p.Brand,
        p.Description,
        pp.Volume,
        pp.Price,
        p.PurchasePrice
),

SalesSummary AS (
    SELECT
        VendorNo,
        VendorName,
        Brand,
        SUM(SalesQuantity) AS TotalSalesQuantity,
        SUM(SalesDollars) AS TotalSalesDollars,
        SUM(ExciseTax) AS TotalExciseTax,
        SUM(SalesPrice) AS TotalSalesPrice
    FROM sales
    GROUP BY
        VendorNo,
        Brand
)

SELECT
    ps.VendorNumber,
    ps.VendorName,
    ps.Brand,
    ps.PurchasePrice,
    ps.Description,
    ps.Volume,
    ps.ActualPrice,
    ps.TotalPurchaseQuantity,
    ps.TotalPurchaseDollars,
    ss.TotalSalesQuantity,
    ss.TotalSalesDollars,
    ss.TotalExciseTax,
    ss.TotalSalesPrice,
    fs.freight_cost
FROM PurchaseSummary ps
LEFT JOIN SalesSummary ss
    ON ps.VendorNumber = ss.VendorNo
    AND ps.Brand = ss.Brand
LEFT JOIN FreightSummary fs
    ON ps.VendorNumber = fs.VendorNumber
ORDER BY TotalPurchaseDollars DESC""", conn);


In [27]:
vendor_sales_summary

,VendorNumber,VendorName,Brand,PurchasePrice,Description,Volume,ActualPrice,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalExciseTax,TotalSalesPrice,freight_cost
0,1128,BROWN-FORMAN CORP,1233,26.27,Jack Daniels No 7 Black,1750,36.99,145080,3811251.60,142049.0,5.101920e+06,260999.20,672819.31,68601.68
1,4425,MARTIGNETTI COMPANIES,3405,23.19,Tito's Handmade Vodka,1750,28.99,164038,3804041.22,160247.0,4.819073e+06,294438.66,561512.37,144929.24
2,17035,PERNOD RICARD USA,8068,18.24,Absolut 80 Proof,1750,24.99,187407,3418303.68,187140.0,4.538121e+06,343854.07,461140.15,123780.22
3,3960,DIAGEO NORTH AMERICA INC,4261,16.17,Capt Morgan Spiced Rum,1750,22.99,201682,3261197.94,200412.0,4.475973e+06,368242.80,420050.01,257032.07
4,3960,DIAGEO NORTH AMERICA INC,3545,21.89,Ketel One Vodka,1750,29.99,138109,3023206.01,135838.0,4.223108e+06,249587.83,545778.28,257032.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,1.32,Concannon Glen Ellen Wh Zin,750,4.99,2,2.64,5.0,1.595000e+01,0.55,10.96,27100.41
10688,8004,SAZERAC CO INC,5683,0.39,Dr McGillicuddy's Apple Pie,50,0.49,6,2.34,134.0,6.566000e+01,7.04,1.47,50293.62
10689,3924,HEAVEN HILL DISTILLERIES,9123,0.74,Deep Eddy Vodka,50,0.99,2,1.48,2.0,1.980000e+00,0.10,0.99,14069.87
10690,3960,DIAGEO NORTH AMERICA INC,6127,1.47,The Club Strawbry Margarita,200,1.99,1,1.47,72.0,1.432800e+02,15.12,77.61,257032.07


This query generate a vendor-wise sales and purchase summary, which is valuable for:

**Performance Optimization:**


*   The query involves heavy joins and aggregations on large datasets like sales and purchase.
*   Storing the pre-aggregated results avoids repeated expensive computations.
*   Helps in analyzing sales,purcahse and pricing for differnt vendors and brand.
* Future benefit of storing this data for faster Dashboarding and Reporting.
* Instead of running expensive and time consuming queries each time, dashboard can take data quickly from vendor_sales_summary.



In [28]:
vendor_sales_summary.dtypes

,0
VendorNumber,int64
VendorName,object
Brand,int64
PurchasePrice,float64
Description,object
Volume,object
ActualPrice,float64
TotalPurchaseQuantity,int64
TotalPurchaseDollars,float64
TotalSalesQuantity,float64


In [29]:
vendor_sales_summary.isnull().sum()

,0
VendorNumber,0
VendorName,0
Brand,0
PurchasePrice,0
Description,0
Volume,0
ActualPrice,0
TotalPurchaseQuantity,0
TotalPurchaseDollars,0
TotalSalesQuantity,178


Checking if there is any inconsistency in obejct data types

In [30]:
vendor_sales_summary['VendorName'].unique()

array(['BROWN-FORMAN CORP          ', 'MARTIGNETTI COMPANIES',
       'PERNOD RICARD USA          ', 'DIAGEO NORTH AMERICA INC   ',
       'BACARDI USA INC            ', 'JIM BEAM BRANDS COMPANY    ',
       'MAJESTIC FINE WINES        ', 'ULTRA BEVERAGE COMPANY LLP ',
       'STOLI GROUP,(USA) LLC      ', 'PROXIMO SPIRITS INC.       ',
       'MOET HENNESSY USA INC      ', 'CAMPARI AMERICA            ',
       'SAZERAC CO INC             ', 'CONSTELLATION BRANDS INC   ',
       'M S WALKER INC             ', 'SAZERAC NORTH AMERICA INC. ',
       'PALM BAY INTERNATIONAL INC ', 'REMY COINTREAU USA INC     ',
       'SIDNEY FRANK IMPORTING CO  ', 'E & J GALLO WINERY         ',
       'WILLIAM GRANT & SONS INC   ', 'HEAVEN HILL DISTILLERIES   ',
       'DISARONNO INTERNATIONAL LLC', 'EDRINGTON AMERICAS         ',
       'CASTLE BRANDS CORP.        ', 'SOUTHERN WINE & SPIRITS NE ',
       'STE MICHELLE WINE ESTATES  ', 'TRINCHERO FAMILY ESTATES   ',
       'MHW LTD                    ', 'W

unnecessary space in vendor_names

In [31]:
#saving vendor_sales_summary data in drive
#vendor_sales_summary.to_csv('/content/drive/MyDrive/vendor_sales_summary.csv', index=False)


In [32]:
vendor_sales_summary['Volume'] = vendor_sales_summary['Volume'].astype(float)
vendor_sales_summary

,VendorNumber,VendorName,Brand,PurchasePrice,Description,Volume,ActualPrice,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalExciseTax,TotalSalesPrice,freight_cost
0,1128,BROWN-FORMAN CORP,1233,26.27,Jack Daniels No 7 Black,1750.0,36.99,145080,3811251.60,142049.0,5.101920e+06,260999.20,672819.31,68601.68
1,4425,MARTIGNETTI COMPANIES,3405,23.19,Tito's Handmade Vodka,1750.0,28.99,164038,3804041.22,160247.0,4.819073e+06,294438.66,561512.37,144929.24
2,17035,PERNOD RICARD USA,8068,18.24,Absolut 80 Proof,1750.0,24.99,187407,3418303.68,187140.0,4.538121e+06,343854.07,461140.15,123780.22
3,3960,DIAGEO NORTH AMERICA INC,4261,16.17,Capt Morgan Spiced Rum,1750.0,22.99,201682,3261197.94,200412.0,4.475973e+06,368242.80,420050.01,257032.07
4,3960,DIAGEO NORTH AMERICA INC,3545,21.89,Ketel One Vodka,1750.0,29.99,138109,3023206.01,135838.0,4.223108e+06,249587.83,545778.28,257032.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,1.32,Concannon Glen Ellen Wh Zin,750.0,4.99,2,2.64,5.0,1.595000e+01,0.55,10.96,27100.41
10688,8004,SAZERAC CO INC,5683,0.39,Dr McGillicuddy's Apple Pie,50.0,0.49,6,2.34,134.0,6.566000e+01,7.04,1.47,50293.62
10689,3924,HEAVEN HILL DISTILLERIES,9123,0.74,Deep Eddy Vodka,50.0,0.99,2,1.48,2.0,1.980000e+00,0.10,0.99,14069.87
10690,3960,DIAGEO NORTH AMERICA INC,6127,1.47,The Club Strawbry Margarita,200.0,1.99,1,1.47,72.0,1.432800e+02,15.12,77.61,257032.07


In [33]:
vendor_sales_summary.fillna(0, inplace=True)

In [34]:
vendor_sales_summary['VendorName'] = vendor_sales_summary['VendorName'].str.strip()

In [35]:
vendor_sales_summary['GrossProfit'] = vendor_sales_summary['TotalSalesDollars'] - vendor_sales_summary['TotalPurchaseDollars']

In [36]:
vendor_sales_summary['GrossProfit'].min()

-52002.78000000001

In [37]:
vendor_sales_summary['ProfitMargin'] = (vendor_sales_summary['GrossProfit'] / vendor_sales_summary['TotalPurchaseDollars']) * 100

In [38]:
vendor_sales_summary['StockTurnover'] = vendor_sales_summary['TotalPurchaseQuantity'] / vendor_sales_summary['TotalSalesQuantity']

In [39]:
vendor_sales_summary['SalesToPurchaseRatio'] = vendor_sales_summary['TotalSalesDollars'] / vendor_sales_summary['TotalPurchaseDollars']

In [40]:
cursor = conn.cursor()